# Import all the libraries required for the Analysis
This is the parent notebook to segment images of the PBMC under compressed and control settings.
We begin with importing all the the necessary libraries used in our analysis. 

In [ ]:
import os
# import cv2  
import numpy as np
from skimage import measure
import imageio
from natsort import natsorted
import subprocess
import scipy.ndimage as ndi
from tifffile import imread,imwrite
import napari
# from scipy.signal import find_peaks,medfilt
# import tifffile
# import math
# from skimage.measure import regionprops,label
# from skimage import filters,segmentation, morphology, exposure
# from scipy.stats import mode
# import subprocess  
# from tqdm import tqdm
# import utilities
import pandas as pd
# from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean

# Segmentation using multiotsu

The approach gives a lot of under segmentation in slices where the images are bright but over segmentation where the images are dim


In [ ]:

folder_path = os.getcwd()
folder_path=os.path.join("..","stacks")

images=[]
path_array=[]



done=[]
for file_name in natsorted(os.listdir(folder_path)):
     if file_name.endswith('.tiff')&file_name.startswith('dapi'):
            done.append(file_name)
            
for file_name in tqdm(done):
        min_size = 800;
        max_size= 20000
        file_path = os.path.join(folder_path, file_name)
        file_path_bin=os.path.join(folder_path,"bw_"+file_name)
        stack=imread(file_path)
        
        if np.shape(np.unique(stack))[0]>10:
        #         for i in range(stack.shape[0]):
        #             stack[i,:,:]=stack[i,:,:]-ndi.gaussian_filter(stack[i,:,:],sigma=(stack.shape[1]/4))

                #threshold_yen=filters.threshold_triangle(restacked_gs_stack)
            threshold_multiotsu=filters.threshold_multiotsu(stack)  

            if min(threshold_multiotsu)>10:
                threshold_multiotsu=min(threshold_multiotsu)
            else:
                threshold_multiotsu=max(threshold_multiotsu)


            binary_stack = stack>threshold_multiotsu    

            for i in range(len(binary_stack)):
                binary_stack[i,:,:]=ndi.binary_fill_holes(binary_stack[i])    

            binary_stack = medfilt(binary_stack, kernel_size=3) 
            labeled_array = label(binary_stack)  
            labeled_array=utilities.filter_regions_by_volume(labeled_array,min_size,max_size)

            for region in regionprops(labeled_array):
                minr, minc, minz, maxr, maxc, maxz = region.bbox

                length = 0.25*(math.sqrt((maxr-minr)**2+(maxc-minc)**2+(4*(maxz-minz))**2))
                labeled_val = np.amax(labeled_array[minr:maxr, minc:maxc, minz:maxz])
                if length > 100:
                    labeled_array[np.where(labeled_array==labeled_val)]=0
        #                 This region meets the criteria to be removed   
        #        binary_stack_1=chanveese_enhancement(stack,binary_stack)
#            tifffile.imwrite(file_path_bin, labeled_array)
        #         print(file_path_bin)        
        #        viewer, image_layer = napari.imshow(stack.astype(float), name='blobs',scale =(4,1,1))
        #        viewer.dims.ndisplay = 3
        #        viewer.add_labels(labeled_array, name='blobs', opacity=0.5,scale =(4,1,1))



# Quality check for drifting and bad segmentation


In [ ]:
folder_path = os.getcwd()
folder_path=os.path.join("..","stacks")

images=[]
path_array=[]

done=[]
for file_name in natsorted(os.listdir(folder_path)):
     if file_name.endswith('.tiff')&file_name.startswith('bw_dapi'):
            done.append(file_name)  
            
file_names=pd.Series(done)            

splitted=file_names.str.split(pat="_",expand=True)
splitted_control=splitted.iloc[np.where(splitted[5]=='12')]
all_patients=splitted_control[4]

print(np.unique(all_patients))
   

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Function to extract colors from a colormap
def get_colors(cmap_name, num_colors=256):
    cmap = plt.get_cmap(cmap_name)
    colors = cmap(np.linspace(0, 1, num_colors))
    return colors

# Choose colormaps to combine
cmap_names = ['jet']

# Extract colors from each colormap
colors = []
for cmap_name in cmap_names:
    colors.extend(get_colors(cmap_name))

# Shuffle the colors
np.random.shuffle(colors)
colors[0]=[0,0,0,1]
print(colors[0])
# Create a new colormap
new_cmap = LinearSegmentedColormap.from_list('new_cmap', colors)
# Test the new colormap
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

fig, ax = plt.subplots(figsize=(8, 2))
ax.imshow(gradient, aspect='auto', cmap=new_cmap)
ax.set_axis_off()

plt.show()

    
    

In [ ]:
folder_path1=os.path.join("..","screenshots")

indexes=splitted.index.values#np.asarray(splitted[splitted[3]=='P66'].index)
patient_fn=file_names.iloc[indexes]

for file in patient_fn:
    print(file)
    fig, axs = plt.subplots(1,2,figsize=(10,5),tight_layout=True)
    seg=imread(os.path.join(folder_path,file))
    stack=imread(os.path.join(folder_path,file[3:]))    
    max_z=np.max(stack,0)
    max_y=np.max(stack,1)   
    max_x=np.max(stack,2)
    max_y = rescale(max_y,(4, 1), anti_aliasing=True)  
    max_x = rescale(max_x,(4, 1), anti_aliasing=True)  
    
    seg_z=np.max(seg,0)
    seg_y=np.max(seg,1)   
    seg_x=np.max(seg,2)
    seg_y = rescale(seg_y,(4, 1), anti_aliasing=True)
    seg_x = rescale(seg_x,(4, 1), anti_aliasing=True)
    
    ax=axs[0]   
    ax.imshow(max_z,cmap="inferno")
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
        
    ax=axs[1]   
    ax.imshow(seg_z,cmap=new_cmap)
    for spine in ax.spines.values():
        spine.set_visible(False)    
    ax.set_xticks([])
    ax.set_yticks([])
        
    plt.suptitle(file)
    plt.show()
    


In [ ]:
    viewer=napari.Viewer()
    viewer.add_image(stack)
    viewer.dims.ndisplay = 3
    viewer.camera.angles = (30, 30, 60)
    viewer.camera.zoom = 2 
    viewer.theme = 'dark'
    viewer.screenshot(os.path.join(folder_path1,file[:-5]+'_img_screenshot.png'))
    viewer.close()

    viewer=napari.Viewer()
    viewer.add_labels(seg)
    viewer.dims.ndisplay = 3
    viewer.camera.angles = (30, 30, 60)
    viewer.camera.zoom = 2 
    viewer.theme = 'dark'
    viewer.screenshot(os.path.join(folder_path1,file[:-5]+'_label_screenshot.png'))
    viewer.close()


In [ ]:
indexes